In [42]:
import torch.nn as nn
import torch, torchvision

class MIL_Atn_Network(nn.Module):
    def __init__(self, num_classes):
        super(MIL_Atn_Network, self).__init__()
        # import timm
        # self.base_model = timm.create_model('resnet18', pretrained=True,  in_chans=1)
        base_model = torchvision.models.resnet18(weights=True)
        state_dict = torch.load('resnet34-b627a593.pth')
        base_model.load_state_dict(state_dict)
        
        self.feature_extractor = nn.Sequential(*list(base_model.children())[:-1])
        self.fc = torch.nn.Linear(base_model.fc.in_features, num_classes)
        self.atn_fc = nn.Linear(base_model.fc.in_features, 1)  # 全连接层用于计算注意力权重

    def  forward(self, x):
        if x.shape[1] == 1:
            x = x.repeat(1, 3, 1, 1, 1)  # 变成3通道
            
        x = x.permute(0, 4, 1, 2, 3)  
        batch_size, patch_z, channel, patch_x, patch_y  = x.size()
        
        x = x.reshape(batch_size*patch_z, channel, patch_x, patch_y) # [batch*patch_z*channel, patch_x, patch_y] # channel = 1
        
        x = self.feature_extractor(x)
    
        x = x.view(batch_size, patch_z, -1)
        
        attention_scores = torch.sigmoid(self.atn_fc(x)) # [batch_size, num_small_images, 1]
        x = torch.sum(x * attention_scores, dim=1)
        print(x.shape)
        x = self.fc(x)

        return x
                                
        
# device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

my_model = MIL_Atn_Network(num_classes=2)

x1 = torch.randn(8, 1, 96, 96, 32)  # batch, channel, x, y, z
output = my_model(x1)

print('output: ', output.shape)


/app/data/anaconda/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


URLError: <urlopen error [Errno 110] Connection timed out>

In [61]:
base_model = torchvision.models.resnet34(weights=False)
state_dict = torch.load('resnet34-b627a593.pth')
base_model.load_state_dict(state_dict)

feature_extractor = nn.Sequential(*list(base_model.children())[:-1])


/app/data/anaconda/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [75]:
(feature_extractor.state_dict()['7.2.conv2.weight'] ==  base_model.state_dict()['layer4.2.conv2.weight']).all()

tensor(True)

In [71]:
feature_extractor.state_dict().keys()

odict_keys(['0.weight', '1.weight', '1.bias', '1.running_mean', '1.running_var', '1.num_batches_tracked', '4.0.conv1.weight', '4.0.bn1.weight', '4.0.bn1.bias', '4.0.bn1.running_mean', '4.0.bn1.running_var', '4.0.bn1.num_batches_tracked', '4.0.conv2.weight', '4.0.bn2.weight', '4.0.bn2.bias', '4.0.bn2.running_mean', '4.0.bn2.running_var', '4.0.bn2.num_batches_tracked', '4.1.conv1.weight', '4.1.bn1.weight', '4.1.bn1.bias', '4.1.bn1.running_mean', '4.1.bn1.running_var', '4.1.bn1.num_batches_tracked', '4.1.conv2.weight', '4.1.bn2.weight', '4.1.bn2.bias', '4.1.bn2.running_mean', '4.1.bn2.running_var', '4.1.bn2.num_batches_tracked', '4.2.conv1.weight', '4.2.bn1.weight', '4.2.bn1.bias', '4.2.bn1.running_mean', '4.2.bn1.running_var', '4.2.bn1.num_batches_tracked', '4.2.conv2.weight', '4.2.bn2.weight', '4.2.bn2.bias', '4.2.bn2.running_mean', '4.2.bn2.running_var', '4.2.bn2.num_batches_tracked', '5.0.conv1.weight', '5.0.bn1.weight', '5.0.bn1.bias', '5.0.bn1.running_mean', '5.0.bn1.running_var', '

In [72]:
 base_model.state_dict().keys()

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.num_batches_tracked', 'layer1.2.conv1.weight', 'layer1.2.bn1.weight', 'layer1.2.bn1.bias', 'layer1.2.bn1.running_mean', 'layer1.2.bn1.running_var', 'layer1.2.bn1.num_batches_tracked', 'layer1.2.conv2.weight', 'layer1.2.bn2.weight', 'layer1.2.bn2.bias', '